## Máster en Big Data y Data Science

### Metodologías de gestión y diseño de proyectos de big data

#### AP2 - Preparación de los datos para clustering

---

En esta libreta se realizan las transforamciones sobre los datasets del escenario considerando su uso para los métodos de clusterización.

---

In [1]:
#Se importan las librerias a utilizar

import pandas as pd

----

##### Lectura de los datasets

Se parte de los mismos datasets con los que se realizó la integración anterior.

In [2]:
df_creditos = pd.read_csv("../../data/processed/datos_creditos_mc.csv", sep=";")
display(df_creditos.head(1))

df_tarjetas = pd.read_csv("../../data/processed/datos_tarjetas_mc.csv", sep=";")
display(df_tarjetas.head(1))

df_integrado = pd.read_csv("../../data/processed/datos_integrados_mc.csv", sep=";")
display(df_tarjetas.head(1))

,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago
0,713061558.0,22,35000,3,123.0,ALQUILER,59000,PERSONAL,0.59,16.02,1,Y


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo
0,713061558.0,36.0,CASADO,ACTIVO,1088.0,M,4010.0,UNIVERSITARIO_COMPLETO,Blue,24.0,2.0


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo
0,713061558.0,36.0,CASADO,ACTIVO,1088.0,M,4010.0,UNIVERSITARIO_COMPLETO,Blue,24.0,2.0


In [3]:
df_integrado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10127 entries, 0 to 10126
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id_cliente             10127 non-null  float64
 1   edad                   10127 non-null  int64  
 2   importe_solicitado     10127 non-null  int64  
 3   duracion_credito       10127 non-null  int64  
 4   antiguedad_empleado    9790 non-null   float64
 5   situacion_vivienda     10127 non-null  object 
 6   ingresos               10127 non-null  int64  
 7   objetivo_credito       10127 non-null  object 
 8   pct_ingreso            10127 non-null  float64
 9   tasa_interes           9215 non-null   float64
 10  estado_credito         10127 non-null  int64  
 11  falta_pago             10127 non-null  object 
 12  completitud_fila_x     10127 non-null  float64
 13  situacion_vivienda_ok  10127 non-null  bool   
 14  antiguedad_cliente     10127 non-null  float64
 15  es

---
#### Aplicación de transformaciones

**Operaciones a realizar**

1. Selección de columnas
2. Filtrado de filas
3. Construcción de atributos
4. Integración de datasets
5. Formateo definitivo


----

Selección de datos

In [4]:
# Se establece qué columnas se eliminan

col_eliminar = ['completitud_fila_x', 'completitud_fila_x','situacion_vivienda_ok','situacion_vivienda_ok','completitud_fila_y','nivel_tarjeta']

# Se ejecuta la operación

df_integrado.drop(col_eliminar, inplace=True, axis=1)

In [5]:
print("Vista del dataset integrado:")
display(df_integrado.head(1))

Vista del dataset integrado:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,...,falta_pago,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,operaciones_ult_12m,personas_a_cargo
0,708082083.0,24,11000,3,5.0,HIPOTECA,64800,INVERSIONES,0.17,5.79,...,N,36.0,CASADO,ACTIVO,15149.0,F,3544.0,SECUNDARIO_COMPLETO,111.0,3.0


Limpieza de los datos (filtrado a nivel de filas)

In [6]:
#Se puede definir una función para aplicar los cálculos
def regla_pct_ingresos_credito(row):
    pct_ingreso = row.pct_ingreso
    ingresos = row.ingresos
    
    if pct_ingreso > 0.5 and ingresos <= 20000:
        # Es un error, no cumple la regla definida
        return 'err'
    else:
        return 'ok'


# Se aplica la función para todos los elementos del dataset
regla_pct_ingresos = df_integrado.apply(lambda row: regla_pct_ingresos_credito(row), axis=1).rename("regla_pct_ingresos")

# Se unen los resultados al dataset inicial
df_integrado = pd.concat([df_integrado, regla_pct_ingresos], axis=1)
df_integrado.head(5)  

,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,...,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,operaciones_ult_12m,personas_a_cargo,regla_pct_ingresos
0,708082083.0,24,11000,3,5.0,HIPOTECA,64800,INVERSIONES,0.17,5.79,...,36.0,CASADO,ACTIVO,15149.0,F,3544.0,SECUNDARIO_COMPLETO,111.0,3.0,ok
1,708083283.0,24,1500,2,0.0,ALQUILER,30996,MEJORAS_HOGAR,0.05,15.99,...,45.0,SOLTERO,PASIVO,992.0,M,3421.0,DESCONOCIDO,21.0,0.0,ok
2,708084558.0,23,10000,2,7.0,OTROS,40104,EDUCACIÓN,0.25,12.72,...,38.0,DIVORCIADO,PASIVO,1447.0,M,8258.0,POSGRADO_COMPLETO,23.0,3.0,ok
3,708085458.0,25,6000,4,2.0,ALQUILER,23198,INVERSIONES,0.26,8.00,...,24.0,SOLTERO,ACTIVO,3940.0,F,1438.3,UNIVERSITARIO_INCOMPLETO,82.0,2.0,ok
4,708086958.0,26,10000,2,0.0,HIPOTECA,50000,EDUCACIÓN,0.20,7.74,...,41.0,CASADO,ACTIVO,4369.0,F,3128.0,UNIVERSITARIO_INCOMPLETO,59.0,2.0,ok


In [7]:
# Se puede definir una función para aplicar los cálculos de la Regla 2
def regla_duracion_ingresos_credito(row):
    duracion_credito = row.duracion_credito
    pct_ingreso = row.pct_ingreso
    situacion_vivienda = row.situacion_vivienda
    
    # Duración mínima permitida
    duracion_minima = df_integrado['duracion_credito'].min()
    
    # Se verifica si la duración del crédito es la mínima permitida
    if duracion_credito == duracion_minima:
        # Se verifica si el porcentaje de ingresos excede el 60%
        if pct_ingreso > 0.6:
            # Si no es propietario de su vivienda, es un error
            if situacion_vivienda != 'PROPIA':
                return 'err'
    return 'ok'

# Se aplica la función para todos los elementos del dataset
regla_duracion_ingresos = df_integrado.apply(lambda row: regla_duracion_ingresos_credito(row), axis=1).rename("regla_duracion_ingresos")

# Se unen los resultados al dataset inicial
df_integrado = pd.concat([df_integrado, regla_duracion_ingresos], axis=1)

# Se visualizan los datos
print("Se visualizan las tuplas que no cumplen con la regla 2:\n")
display(df_integrado[df_integrado.regla_duracion_ingresos == 'err'].head())

# Se verifica la cantidad de elementos
aux_regla_2 = df_integrado[df_integrado.regla_duracion_ingresos == 'err']
print(f"Cantidad de filas que no cumplen la regla 2: {aux_regla_2.shape[0]}")

Se visualizan las tuplas que no cumplen con la regla 2:



,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,...,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,operaciones_ult_12m,personas_a_cargo,regla_pct_ingresos,regla_duracion_ingresos
466,709040508.0,21,15000,2,0.0,HIPOTECA,19500,EDUCACIÓN,0.77,9.64,...,CASADO,ACTIVO,2244.0,M,19802.0,UNIVERSITARIO_INCOMPLETO,58.0,4.0,err,err
541,709186983.0,23,20000,2,1.0,ALQUILER,32900,EDUCACIÓN,0.61,16.00,...,CASADO,ACTIVO,1536.0,M,8714.0,UNIVERSITARIO_INCOMPLETO,38.0,4.0,ok,err
1646,711399408.0,24,29100,2,8.0,HIPOTECA,46000,PERSONAL,0.63,12.99,...,DIVORCIADO,PASIVO,2462.0,F,12373.0,UNIVERSITARIO_COMPLETO,40.0,2.0,ok,err
2573,713115483.0,22,7000,2,0.0,ALQUILER,10000,MEJORAS_HOGAR,0.70,12.21,...,CASADO,PASIVO,2275.0,F,1684.0,UNIVERSITARIO_INCOMPLETO,45.0,3.0,err,err
3041,713962233.0,22,30000,2,1.0,ALQUILER,48000,EDUCACIÓN,0.63,18.39,...,CASADO,ACTIVO,1877.0,F,3035.0,UNIVERSITARIO_COMPLETO,37.0,3.0,ok,err


Cantidad de filas que no cumplen la regla 2: 7


In [8]:
# Se filtran las filas con algún error detectado en edad
print(f"Filas antes del filtro: {df_integrado.shape[0]}")

temp = df_integrado[df_integrado['edad'] < 90]

# Otro filtro posible: por la regla de negocio agregada

temp_c = temp[temp['regla_pct_ingresos'] == 'ok']

# Se filtran las filas por la regla 2

temp_a = temp_c[temp_c['regla_duracion_ingresos'] == 'ok']

# Se filtran las filas por nulos "antiguedad_cliente"

temp_b = temp_a[temp_a['antiguedad_empleado'].notnull()]

# Se filtran las filas con algún error detectado "antiguedad_cliente"

temp_d = temp_a[temp_a['antiguedad_empleado'] < 50]

# Se filtran las filas por nulos "tasa_interes"

df_integrado = temp_d[temp_d['tasa_interes'].notnull()]

print(f"Filas después del filtro: {df_integrado.shape[0]}")

Filas antes del filtro: 10127
Filas después del filtro: 8878


In [9]:
df_integrado.drop('regla_pct_ingresos', inplace=True, axis=1)
df_integrado.drop('regla_duracion_ingresos', inplace=True, axis=1)
df_integrado.drop('id_cliente', inplace=True, axis=1)

C:\Users\Paula Domski\AppData\Local\Temp\ipykernel_17112\3445857298.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_integrado.drop('regla_pct_ingresos', inplace=True, axis=1)
C:\Users\Paula Domski\AppData\Local\Temp\ipykernel_17112\3445857298.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_integrado.drop('regla_duracion_ingresos', inplace=True, axis=1)
C:\Users\Paula Domski\AppData\Local\Temp\ipykernel_17112\3445857298.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gui

Integración de datos

In [10]:
print(f"Cantidad de columnas del dataset integrado: {df_integrado.shape[1]}")

Cantidad de columnas del dataset integrado: 20


#### Transformación de atributos

Observación general del dataset

In [11]:
df_integrado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8878 entries, 0 to 10126
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   edad                 8878 non-null   int64  
 1   importe_solicitado   8878 non-null   int64  
 2   duracion_credito     8878 non-null   int64  
 3   antiguedad_empleado  8878 non-null   float64
 4   situacion_vivienda   8878 non-null   object 
 5   ingresos             8878 non-null   int64  
 6   objetivo_credito     8878 non-null   object 
 7   pct_ingreso          8878 non-null   float64
 8   tasa_interes         8878 non-null   float64
 9   estado_credito       8878 non-null   int64  
 10  falta_pago           8878 non-null   object 
 11  antiguedad_cliente   8878 non-null   float64
 12  estado_civil         8878 non-null   object 
 13  estado_cliente       8878 non-null   object 
 14  gastos_ult_12m       8878 non-null   float64
 15  genero               8878 non-null   objec

##### Procesamiento de valores nulos

Me salto este paso porque yo ya lo realicé en el script de transformación inicial. 

##### Procesamiento de atributos nominales

Se realiza una binarización por aquellos métodos que no pueden operar con atributos no numéricos.

In [12]:
data = pd.get_dummies(df_integrado)
data.head()

,edad,importe_solicitado,duracion_credito,antiguedad_empleado,ingresos,pct_ingreso,tasa_interes,estado_credito,antiguedad_cliente,gastos_ult_12m,...,estado_cliente_ACTIVO,estado_cliente_PASIVO,genero_F,genero_M,nivel_educativo_DESCONOCIDO,nivel_educativo_POSGRADO_COMPLETO,nivel_educativo_POSGRADO_INCOMPLETO,nivel_educativo_SECUNDARIO_COMPLETO,nivel_educativo_UNIVERSITARIO_COMPLETO,nivel_educativo_UNIVERSITARIO_INCOMPLETO
0,24,11000,3,5.0,64800,0.17,5.79,0,36.0,15149.0,...,True,False,True,False,False,False,False,True,False,False
1,24,1500,2,0.0,30996,0.05,15.99,1,45.0,992.0,...,False,True,False,True,True,False,False,False,False,False
2,23,10000,2,7.0,40104,0.25,12.72,0,38.0,1447.0,...,False,True,False,True,False,True,False,False,False,False
3,25,6000,4,2.0,23198,0.26,8.00,0,24.0,3940.0,...,True,False,True,False,False,False,False,False,False,True
4,26,10000,2,0.0,50000,0.20,7.74,0,41.0,4369.0,...,True,False,True,False,False,False,False,False,False,True


Se exportan los resultados para poder utilizarlos en las operaciones de clustering

In [13]:
data.to_csv("../../data/final/datos_clusterizacion.csv", sep=';', index=False)

In [14]:
print(f"Cantidad de filas del dataset integrado: {data.shape[0]}")
print(f"Cantidad de columnas del dataset integrado: {data.shape[1]}")

Cantidad de filas del dataset integrado: 8878
Cantidad de columnas del dataset integrado: 39
